In [ ]:
import threading
import time
import cv2
import numpy as np
import socket

class camera():
    def __init__(self):
        self.face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
        #self.eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')
        self.cap = cv2.VideoCapture(0)
        self.cap.set(3,640)
        self.cap.set(4,480)
        self.a = 0.0
        self.b = 0.0
    def read(self):
        ret, img = self.cap.read()
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        mini = cv2.resize(gray, (int(gray.shape[1]/5 ), int(gray.shape[0]/5)))
        faces = self.face_cascade.detectMultiScale(mini)
        for i in range(len(faces)):
            face_i = faces[i]
            (x, y, w, h) = [v * 5 for v in face_i]
            self.a = 640 -(x+w/2)
            self.b = y + h/2 
            break
    def release(self):
        self.cap.release()
    def printcords(self):
        print(str(self.a) + " " + str(self.b))
    def getcoords(self):
        return (self.a, self.b)
class myThread (threading.Thread):
    def __init__(self, threadID, name, camera):
        threading.Thread.__init__(self)
        self.threadID = threadID
        self.name = name      
        self.cam = camera
    def run(self):
        print("Starting " + self.name)        
        while(not done.is_set()):
            self.cam.read()
        print("The main thread has completed!")
        self.cam.release()
        print("The thread has released the camera!")

# initialize values for thread use
cam = camera()
done = threading.Event()
threads = []

# Create new threads
thread1 = myThread(1, "Thread-1", cam)
thread1.start()

# initialize values for server use
host = ''
port = 8081
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.bind((host, port))
sock.listen(1)
csock, caddr = sock.accept()
print("Conection from: " + str(caddr))

while True:
    req = csock.recv(1024)
    req = req.decode()
    #print(req)
    a, b = cam.getcoords()
    message = str(a) + " " + str(b) + " "
    #print(str(a) + " " + str(b))
    csock.send(message.encode())
    if req == "Close":
        break

# shutting down server and closing socket connections
csock.close()
sock.shutdown(socket.SHUT_RDWR)
sock.close()

# Shutting off threads
done.set()
threads.append(thread1)

# Wait for all threads to complete
for t in threads:
    t.join()
print ("Server is shutdown!")



#threadLock = threading.Lock()
#threads.append(thread2)
# Get lock to synchronize threads
#threadLock.acquire()


        #faces = self.face_cascade.detectMultiScale(gray, 1.3, 5)
        #for (x, y, w, h) in faces:
         #   self.a = 640-(x+w/2)
          #  self.b = y + h/2 
           # break
        # this method took 17.5 seconds to complete. 
        # Around 3.5 frames can be processed per second
#        return (self.a,self.b)            

        # might not actually need a lock since there isn't a dependency
        # Free lock to release next thread
        #threadLock.release()


In [ ]:
"""
while True:
    csock, caddr = sock.accept()
    print("Connection ")
x = 0
while(x < 500):
    time.sleep(0.1)
    x += 1
    a,b = cam.getcoords()
    print(str(a) + " " + str(b))
done.set()

# Add threads to thread list
threads.append(thread1)
"""